In [8]:
import os
import glob
#import WAPORWA modules
os.chdir(r'E:\_WA\WAPORWA\modules') #change to modules path
import WaPOR
import WA
from WA.pickle_basin import pickle_in,pickle_out
from WA.create_NC import main as make_netcdf
from WA.create_NC import _get_lats_lons 
import gdal
import tempfile

ImportError: DLL load failed: The specified module could not be found.

In [6]:
import os
import glob
os.chdir(r'E:\_WA\WAPORWA\modules') #change to modules path
import WaPOR
import WA

ImportError: DLL load failed: The specified module could not be found.

# BASIN Info

In [2]:
INPUT_FOLDER=r"D:\Github\WAPORWA\data\Litani\Input"
BASIN={'Name':'Litani',   
        'Dir':   r"D:\Github\WAPORWA\data\Litani\Main",
        'time_range':['2009-01-01','2018-12-31'],
        'end_month':'DEC',        
        'geo_data':{
                   'basin':r"D:\Github\WAPORWA\data\Global\Basins\BasinLitani.shp",
                      },
        'global_data':{
                     'grace':r"D:\Github\WAPORWA\data\Global\GRACE\GSFC.glb.200301_201607_v02.4-ICE6G",
                     'grand':r"D:\Github\WAPORWA\data\Global\GRanD\GRanD_reservoirs_v1_1.shp",
                     'wdpa':r"D:\Github\WAPORWA\data\Global\WDPA\WDPA_17countries.shp",
                     },
        'input_data':{
                      'yearly':{
                              'lcc':[r""+os.path.join(INPUT_FOLDER,'L2_LCC_A'),
                                     '-','Landcover Class'],
                                'lu':[r"D:\Github\WAPORWA\data\Litani\Main\data\luwa",
                                     '-','WA+ Landuse Categories'],
                              },
                      'monthly':{
                              'p':[r""+os.path.join(INPUT_FOLDER,'L1_PCP_M'),
                                   'mm/month','Precipitation'],
                              'et':[r""+os.path.join(INPUT_FOLDER,'L2_AETI_M'),
                                    'mm/month','Actual Evapotranspiration'],
                              'ret':[r""+os.path.join(INPUT_FOLDER,'L1_RET_M'),
                                    'mm/month','Reference Evapotranspiration'],
                              'i':[r""+os.path.join(INPUT_FOLDER,'L2_I_M'),
                                    'mm/month','Interception'],
                              'nRD':[r""+os.path.join(INPUT_FOLDER,'Rainy_Days'),
                                    'days/month','Number of Rainy Days'],
                              },  
                    'stat':{
                             'thetasat': [r"D:\Github\WAPORWA\data\Global\HiHydroSoils\thetasat_topsoil.tif",
                                          '%','Top-soil Saturated Water Content']
                            }                                                   
                              },
        'input_ts':{
                'dS':None,
                'Qoutlet':r"D:\Github\WAPORWA\data\Litani\Input\Qoutlet_litani_QASMIYE_SeaMouth.csv",
                'Qswout':r"D:\Github\WAPORWA\data\Litani\Input\Qibt_litani_MarkabaTunnel.csv",
                'Qgwout':None,
                
                    },
        'main_data':{
                'yearly':{},
                     'monthly':{},
                     'stat':{},
                     }                
                }

pickle_out(BASIN)

'D:\\Github\\WAPORWA\\data\\Litani\\Main\\Info_20200717_14h48.pickle'

# Create netcdf

In [3]:
#Get inputs for create_NC
cutline=BASIN['geo_data']['basin']
Dir_out=os.path.join(BASIN['Dir'],'data','nc')
if not os.path.exists(Dir_out):
    os.makedirs(Dir_out)    
template=glob.glob(os.path.join(BASIN['input_data']['yearly']['lcc'][0],'*.tif'))[0]          
    

In [4]:
### yearly maps
for key in BASIN['input_data']['yearly']:             
    if BASIN['input_data']['yearly'][key] is not None:
        nc_fn=os.path.join(Dir_out,key+'_yearly.nc')        
        dataset={key:[BASIN['input_data']['yearly'][key][0],
                       ('time','latitude', 'longitude'), 
                       {'units': BASIN['input_data']['yearly'][key][1],                                 
                        'quantity':BASIN['input_data']['yearly'][key][2],
                        'source': 'WaPOR', 'period':'year'}]}
        succes=make_netcdf(nc_fn,BASIN['Name'],dataset,template,cutline,step='year')
        if succes:
            BASIN['main_data']['yearly'][key]=nc_fn
            print('Finished {0}_yearly.nc'.format(key))
            
### monthly maps  
for key in BASIN['input_data']['monthly']:           
    if BASIN['input_data']['monthly'][key] is not None:
        nc_fn=os.path.join(Dir_out,key+'_monthly.nc')                
        dataset={key:[BASIN['input_data']['monthly'][key][0],
                       ('time','latitude', 'longitude'), 
                       {'units': BASIN['input_data']['monthly'][key][1],                                 
                        'quantity':BASIN['input_data']['monthly'][key][2],
                        'source': 'WaPOR', 'period':'month'}]}
        succes=make_netcdf(nc_fn,BASIN['Name'],dataset,template,cutline,step='month')
        if succes:
            BASIN['main_data']['monthly'][key]=nc_fn
            print('Finished {0}_monthly.nc'.format(key))
  
### static maps
for key in BASIN['input_data']['stat']:            
    if BASIN['input_data']['stat'][key] is not None:  
        nc_fn=os.path.join(Dir_out,key+'_stat.nc')                
        dataset={key:[BASIN['input_data']['stat'][key][0],
                       ('latitude', 'longitude'), 
                       {'units': BASIN['input_data']['stat'][key][1],                                 
                        'quantity':BASIN['input_data']['stat'][key][2],
                        'source': 'WaPOR', 'period':'stat'}]}
        succes=make_netcdf(nc_fn,BASIN['Name'],dataset,template,cutline)
        if succes:
            BASIN['main_data']['stat'][key]=nc_fn
            print('Finished {0}stat.nc'.format(key))

pickle_out(BASIN)   

6.806621238863997e-16
-6.806621238863997e-16


100%|█████████████████████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.53it/s]


Finished lcc_yearly.nc
6.806621238863997e-16
-6.806621238863997e-16


100%|█████████████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.15it/s]


Finished lu_yearly.nc
6.806621238863997e-16
-6.806621238863997e-16


...

100%|███████████████████████████████████████████████████████████████████████| 120/120 [00:35<00:00,  3.40it/s]


Finished nRD_monthly.nc
6.806621238863997e-16
-6.806621238863997e-16
{'Top-soil Saturated Water Content': array([[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       ...,
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.]],
      dtype=float32)}
D:\Github\WAPORWA\data\Global\Basins\BasinLitani.shp
Top-soil Saturated Water Content
(958, 1181)


0it [00:00, ?it/s]


Finished thetasatstat.nc


'D:\\Github\\WAPORWA\\data\\Litani\\Main\\Info_20200717_14h52.pickle'